In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, Imputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import lightgbm as lgb

import matplotlib.pyplot as plt

%matplotlib inline

In [47]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import gc

In [2]:
app_train = pd.read_csv("data/application_train.csv")
app_test = pd.read_csv("data/application_test.csv")
bureau =  pd.read_csv("data/bureau.csv")
bureau_bal = pd.read_csv("data/bureau_balance.csv")
credit = pd.read_csv("data/credit_card_balance.csv")
repayments =  pd.read_csv("data/installments_payments.csv")
pos_cash =  pd.read_csv("data/pos_cash_balance.csv")
previous_app =  pd.read_csv("data/previous_application.csv")

In [4]:
app_train.head(5)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
app_train.shape

(307511, 122)

### Exploration

In [6]:
app_train['TARGET'].value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

In [7]:
# Variable types
app_train.dtypes.value_counts()

float64    65
int64      41
object     16
dtype: int64

In [7]:
# Categorial to numerical variables
app_train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

NAME_CONTRACT_TYPE             2
CODE_GENDER                    3
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
NAME_TYPE_SUITE                7
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
OCCUPATION_TYPE               18
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
WALLSMATERIAL_MODE             7
EMERGENCYSTATE_MODE            2
dtype: int64

In [3]:
# Two categories 
le = LabelEncoder()

for col in app_train:
    if app_train[col].dtype == 'object':
        if len(list(app_train[col].unique())) <= 2:
            le.fit(app_train[col])
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])


In [4]:
# Multiple categories

app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

In [10]:
print(app_train.shape)
print(app_test.shape)

(307511, 243)
(48744, 239)


In [5]:
# Align train + test

train_labels = app_train['TARGET']
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)
app_train['TARGET'] = train_labels
print(app_train.shape)
print(app_test.shape)

(307511, 240)
(48744, 239)


In [12]:
# Check for weird things in describe

app_train['AMT_CREDIT'].describe()


count    3.075110e+05
mean     5.990260e+05
std      4.024908e+05
min      4.500000e+04
25%      2.700000e+05
50%      5.135310e+05
75%      8.086500e+05
max      4.050000e+06
Name: AMT_CREDIT, dtype: float64

In [13]:
(app_train['DAYS_BIRTH'] / -365).describe()

count    307511.000000
mean         43.936973
std          11.956133
min          20.517808
25%          34.008219
50%          43.150685
75%          53.923288
max          69.120548
Name: DAYS_BIRTH, dtype: float64

In [14]:
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

In [15]:
# Correlations

corr = app_train.corr()['TARGET'].sort_values()
print(corr.tail(10))
print(corr.head(10))

REG_CITY_NOT_WORK_CITY         0.050994
DAYS_ID_PUBLISH                0.051457
CODE_GENDER_M                  0.054713
DAYS_LAST_PHONE_CHANGE         0.055218
NAME_INCOME_TYPE_Working       0.057481
REGION_RATING_CLIENT           0.058899
REGION_RATING_CLIENT_W_CITY    0.060893
DAYS_EMPLOYED                  0.074958
DAYS_BIRTH                     0.078239
TARGET                         1.000000
Name: TARGET, dtype: float64
EXT_SOURCE_3                           -0.178919
EXT_SOURCE_2                           -0.160472
EXT_SOURCE_1                           -0.155317
NAME_EDUCATION_TYPE_Higher education   -0.056593
CODE_GENDER_F                          -0.054704
NAME_INCOME_TYPE_Pensioner             -0.046209
ORGANIZATION_TYPE_XNA                  -0.045987
FLOORSMAX_AVG                          -0.044003
FLOORSMAX_MEDI                         -0.043768
FLOORSMAX_MODE                         -0.043226
Name: TARGET, dtype: float64


In [16]:
c = app_train.corr().abs()
c1 = c.unstack()
c2 = c1.sort_values(kind="quicksort", na_position="first").drop_duplicates()

print(c2.tail(10))

ENTRANCES_MEDI            ENTRANCES_AVG                 0.996886
FLOORSMAX_AVG             FLOORSMAX_MEDI                0.997034
FLOORSMIN_MEDI            FLOORSMIN_AVG                 0.997241
OBS_60_CNT_SOCIAL_CIRCLE  OBS_30_CNT_SOCIAL_CIRCLE      0.998490
YEARS_BUILD_AVG           YEARS_BUILD_MEDI              0.998495
FLAG_EMP_PHONE            NAME_INCOME_TYPE_Pensioner    0.999515
ORGANIZATION_TYPE_XNA     NAME_INCOME_TYPE_Pensioner    0.999648
                          FLAG_EMP_PHONE                0.999868
CODE_GENDER_F             CODE_GENDER_M                 0.999971
SK_ID_CURR                SK_ID_CURR                    1.000000
dtype: float64


In [ ]:
# Single correlation checks

print(app_train['EXT_SOURCE_1'].corr(app_train['EXT_SOURCE_2']))
print(app_train['EXT_SOURCE_2'].corr(app_train['EXT_SOURCE_3']))
print(app_train['EXT_SOURCE_1'].corr(app_train['EXT_SOURCE_3']))

In [ ]:
# Missing values - checking individual columns

nulls = app_train['EXT_SOURCE_1'].isnull().sum()
nulls/307511

In [ ]:
# Missing values - fill in with average

train_x = app_train.copy()
train_x = train_x.drop('TARGET', axis=1)
test = app_test.copy()
imputer = Imputer(strategy = 'mean')
imputer.fit(train_x)
train_x = imputer.transform(train_x)
test = imputer.transform(test)

In [ ]:
x = app_train.copy()
x = x.drop('TARGET', axis = 1)

### Simple logistic regression

In [ ]:
lr = LogisticRegression()
lr.fit(train_x, train_labels)
predictions = lr.predict_proba(test)[:, 1]

### Simple random forest

In [ ]:
rf = RandomForestClassifier(n_estimators = 100)
rf.fit(train_x, train_labels)
predictions = rf.predict_proba(test)[:, 1]

In [ ]:
# Feature importances

importance_values = rf.feature_importances_
cols = x.columns
features = list(cols)
feature_importances = pd.DataFrame({'feature': features, 'importance': importance_values})


In [ ]:
feature_importances = feature_importances.sort_values('importance', ascending = False).reset_index()
feature_importances.head(10)

### Adding in other data - previous applications

In [5]:
previous_app.shape

(1670214, 37)

In [13]:
previous_app.dtypes.value_counts()

object     16
float64    15
int64       6
dtype: int64

In [19]:
previous_app.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

NAME_CONTRACT_TYPE              4
WEEKDAY_APPR_PROCESS_START      7
FLAG_LAST_APPL_PER_CONTRACT     2
NAME_CASH_LOAN_PURPOSE         25
NAME_CONTRACT_STATUS            4
NAME_PAYMENT_TYPE               4
CODE_REJECT_REASON              9
NAME_TYPE_SUITE                 7
NAME_CLIENT_TYPE                4
NAME_GOODS_CATEGORY            28
NAME_PORTFOLIO                  5
NAME_PRODUCT_TYPE               3
CHANNEL_TYPE                    8
NAME_SELLER_INDUSTRY           11
NAME_YIELD_GROUP                5
PRODUCT_COMBINATION            17
dtype: int64

In [6]:
prev_app_count = previous_app.groupby('SK_ID_CURR', as_index=False)['SK_ID_PREV'].count().rename(columns = {'SK_ID_PREV': 'previous_app_count'})
prev_app_count.head()

,SK_ID_CURR,previous_app_count
0,100001,1
1,100002,1
2,100003,3
3,100004,1
4,100005,2


In [7]:
train2 = app_train.copy()
train2 = train2.merge(prev_app_count, on = 'SK_ID_CURR', how = 'left')
train2['previous_app_count'] = train2['previous_app_count'].fillna(0)
train2.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,TARGET,previous_app_count
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,0,0,0,0,1,0,1,0,1,1.0
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,0,0,0,0,0,0,1,0,0,3.0
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,0,0,0,0,0,0,0,0,0,1.0
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,0,0,0,0,0,0,0,0,0,9.0
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,0,0,0,0,0,0,0,0,0,6.0


In [8]:
# Stats for numerical variables in previous application dataset

prev_app_stats = previous_app.drop(columns = ['SK_ID_PREV']).groupby('SK_ID_CURR', as_index = False).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
prev_app_stats.head()

SK_ID_CURR AMT_ANNUITY                                              \
                   count       mean        max       min         sum   
0     100001           1   3951.000   3951.000  3951.000    3951.000   
1     100002           1   9251.775   9251.775  9251.775    9251.775   
2     100003           3  56553.990  98356.995  6737.310  169661.970   
3     100004           1   5357.250   5357.250  5357.250    5357.250   
4     100005           1   4813.200   4813.200  4813.200    4813.200   

  AMT_APPLICATION                                ...  DAYS_TERMINATION  \
            count       mean       max       min ...             count   
0               1   24835.50   24835.5   24835.5 ...                 1   
1               1  179055.00  179055.0  179055.0 ...                 1   
2               3  435436.50  900000.0   68809.5 ...                 3   
3               1   24282.00   24282.0   24282.0 ...                 1   
4               2   22308.75   44617.5       0.0 ...                 1   

                                       NFLAG_INSURED_ON_APPROVAL            \
          mean     max     min     sum                     count      mean   
0 -1612.000000 -1612.0 -1612.0 -1612.0                         1  0.000000   
1   -17.000000   -17.0   -17.0   -17.0                         1  0.000000   
2 -1047.333333  -527.0 -1976.0 -3142.0                         3  0.666667   
3  -714.000000  -714.0  -714.0  -714.0                         1  0.000000   
4  -460.000000  -460.0  -460.0  -460.0                         1  0.000000   

                  
   max  min  sum  
0  0.0  0.0  0.0  
1  0.0  0.0  0.0  
2  1.0  0.0  2.0  
3  0.0  0.0  0.0  
4  0.0  0.0  0.0  

[5 rows x 96 columns]

In [9]:
# Rename columns
def rename_cols(df):
    columns = ['SK_ID_CURR']
    
    for col in df.columns.levels[0]:
        if col != 'SK_ID_CURR':
            for stat in df.columns.levels[1][:-1]:
                columns.append('prev_app_'+col+'_'+stat)
                
    df.columns = columns
    return df

In [10]:
prev_app_stats = rename_cols(prev_app_stats)
prev_app_stats.head()

,SK_ID_CURR,prev_app_AMT_ANNUITY_count,prev_app_AMT_ANNUITY_mean,prev_app_AMT_ANNUITY_max,prev_app_AMT_ANNUITY_min,prev_app_AMT_ANNUITY_sum,prev_app_AMT_APPLICATION_count,prev_app_AMT_APPLICATION_mean,prev_app_AMT_APPLICATION_max,prev_app_AMT_APPLICATION_min,...,prev_app_DAYS_TERMINATION_count,prev_app_DAYS_TERMINATION_mean,prev_app_DAYS_TERMINATION_max,prev_app_DAYS_TERMINATION_min,prev_app_DAYS_TERMINATION_sum,prev_app_NFLAG_INSURED_ON_APPROVAL_count,prev_app_NFLAG_INSURED_ON_APPROVAL_mean,prev_app_NFLAG_INSURED_ON_APPROVAL_max,prev_app_NFLAG_INSURED_ON_APPROVAL_min,prev_app_NFLAG_INSURED_ON_APPROVAL_sum
0,100001,1,3951.000,3951.000,3951.000,3951.000,1,24835.50,24835.5,24835.5,...,1,-1612.000000,-1612.0,-1612.0,-1612.0,1,0.000000,0.0,0.0,0.0
1,100002,1,9251.775,9251.775,9251.775,9251.775,1,179055.00,179055.0,179055.0,...,1,-17.000000,-17.0,-17.0,-17.0,1,0.000000,0.0,0.0,0.0
2,100003,3,56553.990,98356.995,6737.310,169661.970,3,435436.50,900000.0,68809.5,...,3,-1047.333333,-527.0,-1976.0,-3142.0,3,0.666667,1.0,0.0,2.0
3,100004,1,5357.250,5357.250,5357.250,5357.250,1,24282.00,24282.0,24282.0,...,1,-714.000000,-714.0,-714.0,-714.0,1,0.000000,0.0,0.0,0.0
4,100005,1,4813.200,4813.200,4813.200,4813.200,2,22308.75,44617.5,0.0,...,1,-460.000000,-460.0,-460.0,-460.0,1,0.000000,0.0,0.0,0.0


In [11]:
train2 = train2.merge(prev_app_stats, on = 'SK_ID_CURR', how = 'left')
train2.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,prev_app_DAYS_TERMINATION_count,prev_app_DAYS_TERMINATION_mean,prev_app_DAYS_TERMINATION_max,prev_app_DAYS_TERMINATION_min,prev_app_DAYS_TERMINATION_sum,prev_app_NFLAG_INSURED_ON_APPROVAL_count,prev_app_NFLAG_INSURED_ON_APPROVAL_mean,prev_app_NFLAG_INSURED_ON_APPROVAL_max,prev_app_NFLAG_INSURED_ON_APPROVAL_min,prev_app_NFLAG_INSURED_ON_APPROVAL_sum
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,1.0,-17.000000,-17.0,-17.0,-17.0,1.0,0.000000,0.0,0.0,0.0
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,3.0,-1047.333333,-527.0,-1976.0,-3142.0,3.0,0.666667,1.0,0.0,2.0
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,1.0,-714.000000,-714.0,-714.0,-714.0,1.0,0.000000,0.0,0.0,0.0
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,4.0,182481.750000,365243.0,-416.0,729927.0,4.0,0.000000,0.0,0.0,0.0
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,5.0,72143.800000,365243.0,-2041.0,360719.0,5.0,0.600000,1.0,0.0,3.0


In [20]:
train2.shape

(307511, 336)

In [12]:
# Categorical variables only
categorical = pd.get_dummies(previous_app.select_dtypes('object'))
categorical['SK_ID_CURR'] = previous_app['SK_ID_CURR']
categorical.head()

,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,...,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest,SK_ID_CURR
0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,271877
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,108129
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,122040
3,1,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,176158
4,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,202054


In [13]:
# Counts for categorical variables in previous app dataset
categorical_counts = categorical.groupby('SK_ID_CURR', as_index=False).agg('sum')
categorical_counts.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,...,PRODUCT_COMBINATION_Cash X-Sell: low,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest
0,100001,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,100002,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,100003,1,2,0,0,1,0,1,1,0,...,1,0,1,0,1,0,0,0,0,0
3,100004,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,100005,1,1,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [14]:
def rename_cols_cat(df):
    columns = ['SK_ID_CURR']
    
    for col in df.columns:
        if col != 'SK_ID_CURR':
            columns.append('prev_app_'+col+'_count')

    df.columns = columns
    return df

In [15]:
categorical_counts = rename_cols_cat(categorical_counts)
categorical_counts.head()

,SK_ID_CURR,prev_app_NAME_CONTRACT_TYPE_Cash loans_count,prev_app_NAME_CONTRACT_TYPE_Consumer loans_count,prev_app_NAME_CONTRACT_TYPE_Revolving loans_count,prev_app_NAME_CONTRACT_TYPE_XNA_count,prev_app_WEEKDAY_APPR_PROCESS_START_FRIDAY_count,prev_app_WEEKDAY_APPR_PROCESS_START_MONDAY_count,prev_app_WEEKDAY_APPR_PROCESS_START_SATURDAY_count,prev_app_WEEKDAY_APPR_PROCESS_START_SUNDAY_count,prev_app_WEEKDAY_APPR_PROCESS_START_THURSDAY_count,...,prev_app_PRODUCT_COMBINATION_Cash X-Sell: low_count,prev_app_PRODUCT_COMBINATION_Cash X-Sell: middle_count,prev_app_PRODUCT_COMBINATION_POS household with interest_count,prev_app_PRODUCT_COMBINATION_POS household without interest_count,prev_app_PRODUCT_COMBINATION_POS industry with interest_count,prev_app_PRODUCT_COMBINATION_POS industry without interest_count,prev_app_PRODUCT_COMBINATION_POS mobile with interest_count,prev_app_PRODUCT_COMBINATION_POS mobile without interest_count,prev_app_PRODUCT_COMBINATION_POS other with interest_count,prev_app_PRODUCT_COMBINATION_POS others without interest_count
0,100001,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,100002,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,100003,1,2,0,0,1,0,1,1,0,...,1,0,1,0,1,0,0,0,0,0
3,100004,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,100005,1,1,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [16]:
train2 = train2.merge(categorical_counts, left_on = 'SK_ID_CURR', right_index = True, how = 'left')
train2.head()

,SK_ID_CURR_x,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,prev_app_PRODUCT_COMBINATION_Cash X-Sell: low_count,prev_app_PRODUCT_COMBINATION_Cash X-Sell: middle_count,prev_app_PRODUCT_COMBINATION_POS household with interest_count,prev_app_PRODUCT_COMBINATION_POS household without interest_count,prev_app_PRODUCT_COMBINATION_POS industry with interest_count,prev_app_PRODUCT_COMBINATION_POS industry without interest_count,prev_app_PRODUCT_COMBINATION_POS mobile with interest_count,prev_app_PRODUCT_COMBINATION_POS mobile without interest_count,prev_app_PRODUCT_COMBINATION_POS other with interest_count,prev_app_PRODUCT_COMBINATION_POS others without interest_count
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,2.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [26]:
train2.shape

(307511, 480)

In [54]:
# Find & remove columns that have >75% missing values

In [17]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_percent = mis_val_percent.sort_values(ascending=False)

    return mis_val_percent

In [75]:
missing = missing_values_table(train2)

In [77]:
missing.head(10)

prev_app_RATE_INTEREST_PRIVILEGED_min     98.501192
prev_app_RATE_INTEREST_PRIVILEGED_max     98.501192
prev_app_RATE_INTEREST_PRIVILEGED_mean    98.501192
prev_app_RATE_INTEREST_PRIMARY_min        98.501192
prev_app_RATE_INTEREST_PRIMARY_max        98.501192
prev_app_RATE_INTEREST_PRIMARY_mean       98.501192
COMMONAREA_MEDI                           69.872297
COMMONAREA_AVG                            69.872297
COMMONAREA_MODE                           69.872297
NONLIVINGAPARTMENTS_MODE                  69.432963
dtype: float64

In [80]:
missing = missing[missing < 75]
missing.shape

(474,)

In [19]:
train2 = remove_missing_values(train2)

(6,)


In [18]:
def remove_missing_values(train3):
    missing = missing_values_table(train3)
    to_remove = missing[missing > 75]
    print(to_remove.shape)
    train3 = train3.drop(to_remove.index, axis=1)
    return train3

In [144]:
def numerical_variables(df):
    group = df.drop(columns = ['SK_ID_PREV']).groupby('SK_ID_CURR', as_index = False).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
    stats = rename_cols(group)
    return stats

def categorical_variables(df):
    categorical = pd.get_dummies(df.select_dtypes('object'))
    categorical['SK_ID_CURR'] = df['SK_ID_CURR']
    categorical_counts = categorical.groupby('SK_ID_CURR', as_index=False).agg('sum')
    categorical_counts = rename_cols_cat(categorical_counts)
    return categorical_counts

def merge(train, new):
    train = train.merge(new, left_on = 'SK_ID_CURR', how = 'left')
    return train

In [20]:
def numericals(df, group_by, col_name):

    for col in df:
        if col != group_by and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_by]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_by] = group_ids

    agg = numeric_df.groupby(group_by).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()

    columns = [group_by]

    for var in agg.columns.levels[0]:
        if var != group_by:
            for stat in agg.columns.levels[1][:-1]:
                columns.append('%s_%s_%s' % (col_name, var, stat))

    agg.columns = columns
    return agg

In [21]:
def categoricals(df, group_by, col_name):

    categorical = pd.get_dummies(df.select_dtypes('object'))
    categorical[group_by] = df[group_by]
    categorical = categorical.groupby(group_by).agg(['sum', 'mean'])
    
    column_names = []
    
    for var in categorical.columns.levels[0]:
        for stat in ['count', 'count_norm']:
            column_names.append('%s_%s_%s' % (col_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical

### Adding in other data - cash, credit, installments

In [22]:
# CASH

cash_num = numericals(pos_cash, "SK_ID_PREV", "cash")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [33]:
cash_num.head()

,SK_ID_PREV,cash_MONTHS_BALANCE_count,cash_MONTHS_BALANCE_mean,cash_MONTHS_BALANCE_max,cash_MONTHS_BALANCE_min,cash_MONTHS_BALANCE_sum,cash_CNT_INSTALMENT_count,cash_CNT_INSTALMENT_mean,cash_CNT_INSTALMENT_max,cash_CNT_INSTALMENT_min,...,cash_SK_DPD_count,cash_SK_DPD_mean,cash_SK_DPD_max,cash_SK_DPD_min,cash_SK_DPD_sum,cash_SK_DPD_DEF_count,cash_SK_DPD_DEF_mean,cash_SK_DPD_DEF_max,cash_SK_DPD_DEF_min,cash_SK_DPD_DEF_sum
0,1000001,3,-9.0,-8,-10,-27,3,8.666667,12.0,2.0,...,3,0.0,0,0,0,3,0.0,0,0,0
1,1000002,5,-52.0,-50,-54,-260,5,5.200000,6.0,4.0,...,5,0.0,0,0,0,5,0.0,0,0,0
2,1000003,4,-2.5,-1,-4,-10,4,12.000000,12.0,12.0,...,4,0.0,0,0,0,4,0.0,0,0,0
3,1000004,8,-25.5,-22,-29,-204,8,9.625000,10.0,7.0,...,8,0.0,0,0,0,8,0.0,0,0,0
4,1000005,11,-51.0,-46,-56,-561,11,10.000000,10.0,10.0,...,11,0.0,0,0,0,11,0.0,0,0,0


In [23]:
cash_cat = categoricals(pos_cash, "SK_ID_PREV", "cash")

In [35]:
cash_cat.head()

,cash_NAME_CONTRACT_STATUS_Active_count,cash_NAME_CONTRACT_STATUS_Active_count_norm,cash_NAME_CONTRACT_STATUS_Amortized debt_count,cash_NAME_CONTRACT_STATUS_Amortized debt_count_norm,cash_NAME_CONTRACT_STATUS_Approved_count,cash_NAME_CONTRACT_STATUS_Approved_count_norm,cash_NAME_CONTRACT_STATUS_Canceled_count,cash_NAME_CONTRACT_STATUS_Canceled_count_norm,cash_NAME_CONTRACT_STATUS_Completed_count,cash_NAME_CONTRACT_STATUS_Completed_count_norm,cash_NAME_CONTRACT_STATUS_Demand_count,cash_NAME_CONTRACT_STATUS_Demand_count_norm,cash_NAME_CONTRACT_STATUS_Returned to the store_count,cash_NAME_CONTRACT_STATUS_Returned to the store_count_norm,cash_NAME_CONTRACT_STATUS_Signed_count,cash_NAME_CONTRACT_STATUS_Signed_count_norm,cash_NAME_CONTRACT_STATUS_XNA_count,cash_NAME_CONTRACT_STATUS_XNA_count_norm
SK_ID_PREV,,,,,,,,,,,,,,,,,,
1000001,2,0.666667,0,0.0,0,0.0,0,0.0,1,0.333333,0,0.0,0,0.0,0,0.0,0,0.0
1000002,4,0.800000,0,0.0,0,0.0,0,0.0,1,0.200000,0,0.0,0,0.0,0,0.0,0,0.0
1000003,4,1.000000,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0
1000004,7,0.875000,0,0.0,0,0.0,0,0.0,1,0.125000,0,0.0,0,0.0,0,0.0,0,0.0
1000005,10,0.909091,0,0.0,0,0.0,0,0.0,1,0.090909,0,0.0,0,0.0,0,0.0,0,0.0


In [24]:
cash_agg = cash_num.merge(cash_cat, right_index = True, left_on = 'SK_ID_PREV', how = 'outer')
cash_agg = pos_cash[['SK_ID_PREV', 'SK_ID_CURR']].merge(cash_agg, on = 'SK_ID_PREV', how = 'left')
cash_agg.head(5)

,SK_ID_PREV,SK_ID_CURR,cash_MONTHS_BALANCE_count,cash_MONTHS_BALANCE_mean,cash_MONTHS_BALANCE_max,cash_MONTHS_BALANCE_min,cash_MONTHS_BALANCE_sum,cash_CNT_INSTALMENT_count,cash_CNT_INSTALMENT_mean,cash_CNT_INSTALMENT_max,...,cash_NAME_CONTRACT_STATUS_Completed_count,cash_NAME_CONTRACT_STATUS_Completed_count_norm,cash_NAME_CONTRACT_STATUS_Demand_count,cash_NAME_CONTRACT_STATUS_Demand_count_norm,cash_NAME_CONTRACT_STATUS_Returned to the store_count,cash_NAME_CONTRACT_STATUS_Returned to the store_count_norm,cash_NAME_CONTRACT_STATUS_Signed_count,cash_NAME_CONTRACT_STATUS_Signed_count_norm,cash_NAME_CONTRACT_STATUS_XNA_count,cash_NAME_CONTRACT_STATUS_XNA_count_norm
0,1803195,182943,33,-18.0,-2,-34,-594,33,48.000000,48.0,...,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0
1,1715348,367990,19,-25.0,-16,-34,-475,19,35.052632,36.0,...,1,0.052632,0,0.0,0,0.0,0,0.0,0,0.0
2,1784872,397406,12,-28.5,-23,-34,-342,12,12.000000,12.0,...,1,0.083333,0,0.0,0,0.0,0,0.0,0,0.0
3,1903291,269225,28,-27.5,-14,-41,-770,28,47.285714,48.0,...,1,0.035714,0,0.0,0,0.0,0,0.0,0,0.0
4,2341044,334279,36,-18.5,-1,-36,-666,36,36.000000,36.0,...,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0


In [25]:
cash_final = numericals(cash_agg.drop(columns = ['SK_ID_PREV']), 'SK_ID_CURR', 'cash')

In [38]:
cash_final.shape

(337252, 216)

In [40]:
train2.shape

(307511, 474)

In [26]:
train2 = train2.rename(columns={'SK_ID_CURR_x':'SK_ID_CURR'})
train2.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,prev_app_PRODUCT_COMBINATION_Cash X-Sell: low_count,prev_app_PRODUCT_COMBINATION_Cash X-Sell: middle_count,prev_app_PRODUCT_COMBINATION_POS household with interest_count,prev_app_PRODUCT_COMBINATION_POS household without interest_count,prev_app_PRODUCT_COMBINATION_POS industry with interest_count,prev_app_PRODUCT_COMBINATION_POS industry without interest_count,prev_app_PRODUCT_COMBINATION_POS mobile with interest_count,prev_app_PRODUCT_COMBINATION_POS mobile without interest_count,prev_app_PRODUCT_COMBINATION_POS other with interest_count,prev_app_PRODUCT_COMBINATION_POS others without interest_count
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,2.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [27]:
train3 = train2.merge(cash_final, on = 'SK_ID_CURR', how = 'left')
train3.shape

(307511, 689)

In [28]:
# CREDIT

credit_num = numericals(credit, "SK_ID_PREV", "credit")
credit_cat = categoricals(credit, "SK_ID_PREV", "credit")

credit_agg = credit_num.merge(credit_cat, right_index = True, left_on = 'SK_ID_PREV', how = 'outer')
credit_agg = credit[['SK_ID_PREV', 'SK_ID_CURR']].merge(credit_agg, on = 'SK_ID_PREV', how = 'left')

credit_final = numericals(credit_agg.drop(columns = ['SK_ID_PREV']), 'SK_ID_CURR', 'credit')

credit_final.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,SK_ID_CURR,credit_credit_MONTHS_BALANCE_count_count,credit_credit_MONTHS_BALANCE_count_mean,credit_credit_MONTHS_BALANCE_count_max,credit_credit_MONTHS_BALANCE_count_min,credit_credit_MONTHS_BALANCE_count_sum,credit_credit_MONTHS_BALANCE_mean_count,credit_credit_MONTHS_BALANCE_mean_mean,credit_credit_MONTHS_BALANCE_mean_max,credit_credit_MONTHS_BALANCE_mean_min,...,credit_credit_NAME_CONTRACT_STATUS_Signed_count_count,credit_credit_NAME_CONTRACT_STATUS_Signed_count_mean,credit_credit_NAME_CONTRACT_STATUS_Signed_count_max,credit_credit_NAME_CONTRACT_STATUS_Signed_count_min,credit_credit_NAME_CONTRACT_STATUS_Signed_count_sum,credit_credit_NAME_CONTRACT_STATUS_Signed_count_norm_count,credit_credit_NAME_CONTRACT_STATUS_Signed_count_norm_mean,credit_credit_NAME_CONTRACT_STATUS_Signed_count_norm_max,credit_credit_NAME_CONTRACT_STATUS_Signed_count_norm_min,credit_credit_NAME_CONTRACT_STATUS_Signed_count_norm_sum
0,100006,6,6.0,6,6,36,6,-3.5,-3.5,-3.5,...,6,0.0,0,0,0.0,6,0.0,0.0,0.0,0.0
1,100011,74,74.0,74,74,5476,74,-38.5,-38.5,-38.5,...,74,0.0,0,0,0.0,74,0.0,0.0,0.0,0.0
2,100013,96,96.0,96,96,9216,96,-48.5,-48.5,-48.5,...,96,0.0,0,0,0.0,96,0.0,0.0,0.0,0.0
3,100021,17,17.0,17,17,289,17,-10.0,-10.0,-10.0,...,17,0.0,0,0,0.0,17,0.0,0.0,0.0,0.0
4,100023,8,8.0,8,8,64,8,-7.5,-7.5,-7.5,...,8,0.0,0,0,0.0,8,0.0,0.0,0.0,0.0


In [29]:
train3 = train3.merge(credit_final, on = 'SK_ID_CURR', how = 'left')
train3.shape

(307511, 1259)

In [30]:
missing = missing_values_table(train3)
missing.head()

credit_credit_AMT_PAYMENT_CURRENT_mean_min    80.1438
credit_credit_AMT_PAYMENT_CURRENT_min_max     80.1438
credit_credit_AMT_PAYMENT_CURRENT_max_mean    80.1438
credit_credit_AMT_PAYMENT_CURRENT_max_max     80.1438
credit_credit_AMT_PAYMENT_CURRENT_mean_max    80.1438
dtype: float64

In [131]:
to_remove = missing[missing > 75]
to_remove.head()

prev_app_RATE_INTEREST_PRIVILEGED_min     98.501192
prev_app_RATE_INTEREST_PRIVILEGED_max     98.501192
prev_app_RATE_INTEREST_PRIVILEGED_mean    98.501192
prev_app_RATE_INTEREST_PRIMARY_min        98.501192
prev_app_RATE_INTEREST_PRIMARY_max        98.501192
dtype: float64

In [132]:
to_remove.shape
train3 = train3.drop(to_remove.index, axis=1)

(69,)

In [138]:
train3.shape

(307511, 1196)

In [31]:
# INSTALLMENTS

instal_num = numericals(repayments, "SK_ID_PREV", "instal")
instal_num = repayments[['SK_ID_PREV', 'SK_ID_CURR']].merge(instal_num, on = 'SK_ID_PREV', how = 'left')

instal_final = numericals(instal_num.drop(columns = ['SK_ID_PREV']), 'SK_ID_CURR', 'instal')

instal_final.head()

,SK_ID_CURR,instal_instal_NUM_INSTALMENT_VERSION_count_count,instal_instal_NUM_INSTALMENT_VERSION_count_mean,instal_instal_NUM_INSTALMENT_VERSION_count_max,instal_instal_NUM_INSTALMENT_VERSION_count_min,instal_instal_NUM_INSTALMENT_VERSION_count_sum,instal_instal_NUM_INSTALMENT_VERSION_mean_count,instal_instal_NUM_INSTALMENT_VERSION_mean_mean,instal_instal_NUM_INSTALMENT_VERSION_mean_max,instal_instal_NUM_INSTALMENT_VERSION_mean_min,...,instal_instal_AMT_PAYMENT_min_count,instal_instal_AMT_PAYMENT_min_mean,instal_instal_AMT_PAYMENT_min_max,instal_instal_AMT_PAYMENT_min_min,instal_instal_AMT_PAYMENT_min_sum,instal_instal_AMT_PAYMENT_sum_count,instal_instal_AMT_PAYMENT_sum_mean,instal_instal_AMT_PAYMENT_sum_max,instal_instal_AMT_PAYMENT_sum_min,instal_instal_AMT_PAYMENT_sum_sum
0,100001,7,3.571429,4,3,25,7,1.142857,1.250000,1.000000,...,7,3963.8250,3980.925,3951.000,27746.775,7,21834.096429,29250.900,11945.025,1.528387e+05
1,100002,19,19.000000,19,19,361,19,1.052632,1.052632,1.052632,...,19,9251.7750,9251.775,9251.775,175783.725,19,219625.695000,219625.695,219625.695,4.172888e+06
2,100003,25,9.160000,12,6,229,25,1.040000,1.142857,1.000000,...,25,46164.3318,98356.995,6662.970,1154108.295,25,453952.220400,1150977.330,80773.380,1.134881e+07
3,100004,3,3.000000,3,3,9,3,1.333333,1.333333,1.333333,...,3,5357.2500,5357.250,5357.250,16071.750,3,21288.465000,21288.465,21288.465,6.386540e+04
4,100005,9,9.000000,9,9,81,9,1.111111,1.111111,1.111111,...,9,4813.2000,4813.200,4813.200,43318.800,9,56161.845000,56161.845,56161.845,5.054566e+05


In [32]:
train3 = train3.merge(instal_final, on = 'SK_ID_CURR', how = 'left')
train3.shape

(307511, 1409)

In [33]:
train3 = remove_missing_values(train3)

(63,)


### Adding in other data - applications from other bureaus

In [149]:
bureau.columns

Index(['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
       'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG',
       'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY'],
      dtype='object')

In [34]:
bureau_num = numericals(bureau, "SK_ID_CURR", "bureau")
bureau_num.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,SK_ID_CURR,bureau_DAYS_CREDIT_count,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_sum,bureau_CREDIT_DAY_OVERDUE_count,bureau_CREDIT_DAY_OVERDUE_mean,bureau_CREDIT_DAY_OVERDUE_max,bureau_CREDIT_DAY_OVERDUE_min,...,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_UPDATE_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_sum,bureau_AMT_ANNUITY_count,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_max,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_sum
0,100001,7,-735.000000,-49,-1572,-5145,7,0.0,0,0,...,7,-93.142857,-6,-155,-652,7,3545.357143,10822.5,0.0,24817.5
1,100002,8,-874.000000,-103,-1437,-6992,8,0.0,0,0,...,8,-499.875000,-7,-1185,-3999,7,0.000000,0.0,0.0,0.0
2,100003,4,-1400.750000,-606,-2586,-5603,4,0.0,0,0,...,4,-816.000000,-43,-2131,-3264,0,NaN,NaN,NaN,0.0
3,100004,2,-867.000000,-408,-1326,-1734,2,0.0,0,0,...,2,-532.000000,-382,-682,-1064,0,NaN,NaN,NaN,0.0
4,100005,3,-190.666667,-62,-373,-572,3,0.0,0,0,...,3,-54.333333,-11,-121,-163,3,1420.500000,4261.5,0.0,4261.5


In [35]:
bureau_cat = categoricals(bureau, "SK_ID_CURR", "bureau")
bureau_cat.head()

,bureau_CREDIT_ACTIVE_Active_count,bureau_CREDIT_ACTIVE_Active_count_norm,bureau_CREDIT_ACTIVE_Bad debt_count,bureau_CREDIT_ACTIVE_Bad debt_count_norm,bureau_CREDIT_ACTIVE_Closed_count,bureau_CREDIT_ACTIVE_Closed_count_norm,bureau_CREDIT_ACTIVE_Sold_count,bureau_CREDIT_ACTIVE_Sold_count_norm,bureau_CREDIT_CURRENCY_currency 1_count,bureau_CREDIT_CURRENCY_currency 1_count_norm,...,bureau_CREDIT_TYPE_Microloan_count,bureau_CREDIT_TYPE_Microloan_count_norm,bureau_CREDIT_TYPE_Mobile operator loan_count,bureau_CREDIT_TYPE_Mobile operator loan_count_norm,bureau_CREDIT_TYPE_Mortgage_count,bureau_CREDIT_TYPE_Mortgage_count_norm,bureau_CREDIT_TYPE_Real estate loan_count,bureau_CREDIT_TYPE_Real estate loan_count_norm,bureau_CREDIT_TYPE_Unknown type of loan_count,bureau_CREDIT_TYPE_Unknown type of loan_count_norm
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3,0.428571,0,0.0,4,0.571429,0,0.0,7,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100002,2,0.250000,0,0.0,6,0.750000,0,0.0,8,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100003,1,0.250000,0,0.0,3,0.750000,0,0.0,4,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100004,0,0.000000,0,0.0,2,1.000000,0,0.0,2,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100005,2,0.666667,0,0.0,1,0.333333,0,0.0,3,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [36]:
train3 = train3.merge(bureau_num, on = 'SK_ID_CURR', how = 'left')

In [37]:
train3 = train3.merge(bureau_cat, on = 'SK_ID_CURR', how = 'left')

In [54]:
train3 = remove_missing_values(train3)

(0,)


In [38]:
bureau_bal_num = numericals(bureau_bal, "SK_ID_BUREAU", "bureau_bal")
bureau_bal_cat = categoricals(bureau_bal, "SK_ID_BUREAU", "bureau_bal")

bureau_bal_agg = bureau_bal_num.merge(bureau_bal_cat, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')
bureau_bal_agg = bureau[['SK_ID_BUREAU', 'SK_ID_CURR']].merge(bureau_bal_agg, on = 'SK_ID_BUREAU', how = 'left')

bureau_bal_final = numericals(bureau_bal_agg.drop(columns = ['SK_ID_BUREAU']), 'SK_ID_CURR', 'bureau_bal')

bureau_bal_final.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,SK_ID_CURR,bureau_bal_bureau_bal_MONTHS_BALANCE_count_count,bureau_bal_bureau_bal_MONTHS_BALANCE_count_mean,bureau_bal_bureau_bal_MONTHS_BALANCE_count_max,bureau_bal_bureau_bal_MONTHS_BALANCE_count_min,bureau_bal_bureau_bal_MONTHS_BALANCE_count_sum,bureau_bal_bureau_bal_MONTHS_BALANCE_mean_count,bureau_bal_bureau_bal_MONTHS_BALANCE_mean_mean,bureau_bal_bureau_bal_MONTHS_BALANCE_mean_max,bureau_bal_bureau_bal_MONTHS_BALANCE_mean_min,...,bureau_bal_bureau_bal_STATUS_X_count_count,bureau_bal_bureau_bal_STATUS_X_count_mean,bureau_bal_bureau_bal_STATUS_X_count_max,bureau_bal_bureau_bal_STATUS_X_count_min,bureau_bal_bureau_bal_STATUS_X_count_sum,bureau_bal_bureau_bal_STATUS_X_count_norm_count,bureau_bal_bureau_bal_STATUS_X_count_norm_mean,bureau_bal_bureau_bal_STATUS_X_count_norm_max,bureau_bal_bureau_bal_STATUS_X_count_norm_min,bureau_bal_bureau_bal_STATUS_X_count_norm_sum
0,100001,7,24.571429,52.0,2.0,172.0,7,-11.785714,-0.5,-25.5,...,7,4.285714,9.0,0.0,30.0,7,0.214590,0.500000,0.0,1.502129
1,100002,8,13.750000,22.0,4.0,110.0,8,-21.875000,-1.5,-39.5,...,8,1.875000,3.0,0.0,15.0,8,0.161932,0.500000,0.0,1.295455
2,100003,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,...,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,0.000000
3,100004,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,...,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,0.000000
4,100005,3,7.000000,13.0,3.0,21.0,3,-3.000000,-1.0,-6.0,...,3,0.666667,1.0,0.0,2.0,3,0.136752,0.333333,0.0,0.410256


In [39]:
train3 = train3.merge(bureau_bal_final, on = 'SK_ID_CURR', how = 'left')
#train3 = remove_missing_values(train3)

In [40]:
train3.shape

(307511, 1557)

In [59]:
train3.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,bureau_bal_bureau_bal_STATUS_X_count_count,bureau_bal_bureau_bal_STATUS_X_count_mean,bureau_bal_bureau_bal_STATUS_X_count_max,bureau_bal_bureau_bal_STATUS_X_count_min,bureau_bal_bureau_bal_STATUS_X_count_sum,bureau_bal_bureau_bal_STATUS_X_count_norm_count,bureau_bal_bureau_bal_STATUS_X_count_norm_mean,bureau_bal_bureau_bal_STATUS_X_count_norm_max,bureau_bal_bureau_bal_STATUS_X_count_norm_min,bureau_bal_bureau_bal_STATUS_X_count_norm_sum
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,8.0,1.875,3.0,0.0,15.0,8.0,0.161932,0.5,0.0,1.295455
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.000000
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.000000
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.000000


### Adjusting test data

In [165]:
app_test.shape

(48744, 239)

In [41]:
test2 = app_test.copy()

test2 = test2.merge(prev_app_stats, on = 'SK_ID_CURR', how = 'left')
test2 = test2.merge(categorical_counts, on = 'SK_ID_CURR', how = 'left')
test2 = test2.merge(cash_final, on = 'SK_ID_CURR', how = 'left')
test2 = test2.merge(credit_final, on = 'SK_ID_CURR', how = 'left')
test2 = test2.merge(instal_final, on = 'SK_ID_CURR', how = 'left')
test2 = test2.merge(bureau_num, on = 'SK_ID_CURR', how = 'left')
test2 = test2.merge(bureau_cat, on = 'SK_ID_CURR', how = 'left')
test2 = test2.merge(bureau_bal_final, on = 'SK_ID_CURR', how = 'left')

test2 = remove_missing_values(test2)

(69,)


In [42]:
test2.shape

(48744, 1554)

In [43]:
train3.shape

(307511, 1557)

In [44]:
# Align

y = train3['TARGET']
train3, test2 = train3.align(test2, join = 'inner', axis = 1)
train3['TARGET'] = y
print(train3.shape)
print(test2.shape)

(307511, 1555)
(48744, 1554)


In [167]:
# Missing values - fill in with average. Done below with pca in pipeline instead

train5 = train4.copy()
train5 = train5.drop('TARGET', axis=1)
test3 = test2.copy()
imputer = Imputer(strategy = 'mean')
imputer.fit(train5)
train5 = imputer.transform(train5)
test3 = imputer.transform(test3)

### Feature selection

In [ ]:
# Remove correlated variables

threshold = 0.9
corr_matrix = train3.corr().abs()
corr_matrix.head()

In [ ]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()

In [ ]:
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
to_drop.shape

In [ ]:
train4 = train3.drop(columns = to_drop)
#test3 = test3.drop(columns = to_drop)

In [ ]:
# PCA

In [44]:
train3 = train3.drop(columns = ['SK_ID_CURR', 'TARGET'])
test2 = test2.drop(columns = ['SK_ID_CURR'])

In [48]:
pipeline = Pipeline(steps = [('imputer', Imputer(strategy = 'mean')),
             ('pca', PCA(n_components=2))])

train_pca = pipeline.fit_transform(train3)
test_pca = pipeline.transform(test2)

In [49]:
train_pca.shape

(307511, 2)

In [47]:
pca = pipeline.named_steps['pca']
print(100 * np.sum(pca.explained_variance_ratio_[:2]))

98.23777448933565


### Models

In [50]:
# Logistic regression comparison

lr3 = LogisticRegression()
lr3.fit(train3, y)
predictions_lr3 = lr3.predict_proba(test2)[:, 1]

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [53]:
# Random forest comparison

rf2 = RandomForestClassifier(n_estimators = 100)
rf2.fit(train_pca, y)

NameError: name 'rf' is not defined

In [54]:
predictions_rf2 = rf2.predict_proba(test_pca)[:, 1]

In [44]:
# Light GBM

model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)


In [61]:
def model(features, test_features, n_folds = 5):
    
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    
    labels = features['TARGET']
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    # deal with categorical variables

    features = pd.get_dummies(features)
    test_features = pd.get_dummies(test_features)
    features, test_features = features.align(test_features, join = 'inner', axis = 1)

    print('Training shape: ', features.shape)
    print('Testing shape: ', test_features.shape)
    

    feature_names = list(features.columns)
    features = np.array(features)
    test_features = np.array(test_features)
    
    # cross validation
    k_fold = KFold(n_splits = n_folds, shuffle = False, random_state = 50)
    
    feature_importance_values = np.zeros(len(feature_names))
    test_predictions = np.zeros(test_features.shape[0])
    out_of_fold = np.zeros(features.shape[0])
    
    valid_scores = []
    train_scores = []
    
    
    for train_indices, valid_indices in k_fold.split(features):
        train_features, train_labels = features[train_indices], labels[train_indices]
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        model = lgb.LGBMClassifier(n_estimators=10000,
                                   objective = 'binary', 
                                   learning_rate = 0.05, 
                                   reg_alpha = 0.1,
                                   reg_lambda = 0.1, 
                                   subsample = 0.8, # quicker if <1
                                   seed = 50)
        
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = 'auto',
                  early_stopping_rounds = 100, verbose = 1)
        
        best_iteration = model.best_iteration_
        
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        

        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        

    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    return submission, feature_importances

In [48]:
submission, fi, metrics = model(train3, test2)

Training Data Shape:  (307511, 1553)
Testing Data Shape:  (48744, 1553)
Training until validation scores don't improve for 100 rounds.
[200]	valid's auc: 0.783149	train's auc: 0.831138
Early stopping, best iteration is:
[289]	valid's auc: 0.784008	train's auc: 0.849237
Training until validation scores don't improve for 100 rounds.
[200]	valid's auc: 0.784606	train's auc: 0.830577
[400]	valid's auc: 0.785599	train's auc: 0.86768
Early stopping, best iteration is:
[313]	valid's auc: 0.785636	train's auc: 0.85317
Training until validation scores don't improve for 100 rounds.
[200]	valid's auc: 0.777356	train's auc: 0.831773
[400]	valid's auc: 0.778849	train's auc: 0.868601
Early stopping, best iteration is:
[370]	valid's auc: 0.779006	train's auc: 0.864193
Training until validation scores don't improve for 100 rounds.
[200]	valid's auc: 0.781604	train's auc: 0.83059
[400]	valid's auc: 0.78256	train's auc: 0.868102
Early stopping, best iteration is:
[316]	valid's auc: 0.78282	train's auc: 

In [55]:
fi = fi.sort_values('importance', ascending = False).reset_index()
print(fi.head(10))

   index                            feature  importance
0     30                       EXT_SOURCE_1       354.0
1     31                       EXT_SOURCE_2       290.0
2     32                       EXT_SOURCE_3       263.8
3      9                         DAYS_BIRTH       200.4
4      5                         AMT_CREDIT       186.8
5      6                        AMT_ANNUITY       179.6
6   1336  instal_instal_AMT_PAYMENT_min_sum       172.6
7      7                    AMT_GOODS_PRICE       139.8
8     10                      DAYS_EMPLOYED       129.8
9   1344             bureau_DAYS_CREDIT_max       106.6


### Submission file

In [55]:
submission = app_test[['SK_ID_CURR']]
submission['TARGET'] = predictions_rf2

submission.head(5)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,SK_ID_CURR,TARGET
0,100001,0.06
1,100005,0.06
2,100013,0.06
3,100028,0.00
4,100038,0.07


In [49]:
submission.to_csv('lgbm.csv', index = False)